In [136]:
import pandas as pd
import datetime
import gzip
import io
import os

---

## <center>Funciones</center>

In [109]:
def extraer_fecha(time):
    time_seconds = time / 1000  # Convertir milisegundos a segundos
    fecha_hora = datetime.datetime.utcfromtimestamp(time_seconds)

    # Eliminar los milisegundos
    fecha_hora = fecha_hora.replace(microsecond=0)

    return fecha_hora


In [110]:
def extraer_fecha_opinion(time):
    
    time = int(time)
    
    time_seconds = time / 1000  # Convertir milisegundos a segundos
    
    fecha_hora = datetime.datetime.utcfromtimestamp(time_seconds)

    # Eliminar los milisegundos
    fecha_hora = fecha_hora.replace(microsecond=0)

    return fecha_hora

---

In [140]:
dicc = {}

for x in range(1,12):
    dicc[x] = pd.read_json(f'datasets\\Google Maps\\reviews-estados\\review-Alabama\\{x}.json', lines=True)
    
    
# Lista para almacenar los DataFrames
lista_dataframes = []

# Iterar sobre los valores del diccionario (los DataFrames)
for df in dicc.values():
    lista_dataframes.append(df)

# Concatenar los DataFrames en uno solo
json = pd.concat(lista_dataframes, ignore_index=True)


In [141]:
''' Elimino la columna pics, las imagenes son muy chicas y no tienen relevancia para su proximo analisis, en cualquier caso si es necesario acceder a las mismas, una alternativa mucho mejor es utilizar la api o scraping con su id'''
json.drop(columns=['pics'],inplace=True) 

In [142]:
''' Utilizo la funcion creada anteriormente, para convertir los milisegundos a una fecha entendible, luego lo separo en 2 columnas para poder comparar el tiempo de respuesta con el de la opinion propiamente dicha'''
json['time'] = json['time'].apply(extraer_fecha)

In [143]:
'''

Desanido la columna resp, para luego poder realizar calculos con el tiempo de respuesta y de ser necesario el analisis de sentimiento de la respuesta por separado.

'''
json['fecha_respuesta'] = None
json['respuesta'] = None

# Desanidar la columna 'resp' si no es None y asignar valores a las nuevas columnas
json.loc[json['resp'].notnull(), 'fecha_respuesta'] = json.loc[json['resp'].notnull(), 'resp'].apply(lambda x: x.get('time') if x else None)
json.loc[json['resp'].notnull(), 'respuesta'] = json.loc[json['resp'].notnull(), 'resp'].apply(lambda x: x.get('text') if x else None)

# Convertir la columna 'respuesta' a tipo de datos 'object' para permitir listas o strings
json['respuesta'] = json['respuesta'].astype('object')

# Luego, realizar la asignación de valores a la columna 'respuesta'
json.loc[json['resp'].notnull(), 'respuesta'] = json.loc[json['resp'].notnull(), 'resp'].apply(lambda x: x.get('text') if x else None)

json.drop(columns=['resp'],inplace=True)

In [144]:
json['fecha_respuesta'].fillna(json['time'], inplace=True)

In [145]:
json['fecha_respuesta'] = json['fecha_respuesta'].astype(str)

In [146]:
# Aplicar la función 'extraer_anio()' a las filas que cumplen con la condición de longitud
json.loc[json['fecha_respuesta'].str.len() < 14, 'fecha_respuesta'] = json.loc[json['fecha_respuesta'].str.len() < 14, 'fecha_respuesta'].apply(extraer_fecha_opinion)
json['fecha_respuesta'] = pd.to_datetime(json['fecha_respuesta'], format='%Y-%m-%d %H:%M:%S')

In [147]:
json2 = json.copy()

json2['anio'] = json['fecha_respuesta'].dt.year  # Extraer el año y crear una nueva columna 'anio'

resultados = json2.groupby('anio').size()  # Agrupar por año y contar la cantidad de registros

resultados

anio
2008         8
2009        12
2010       133
2011       524
2012       770
2013      1346
2014      1729
2015      6117
2016     42198
2017    198758
2018    385741
2019    513182
2020    322927
2021    176555
dtype: int64

In [149]:
json

,user_id,name,time,rating,text,gmap_id,fecha_respuesta,respuesta
0,1.140438e+20,Kanisha Mixon,2020-08-11 17:51:12,5,Very Personable staff! Beautiful and clean env...,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,2020-08-11 17:51:12,None
1,1.160090e+20,Brandie Hodges,2021-01-06 02:10:39,5,Best clothing intown,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,2021-01-06 02:10:39,None
2,1.062399e+20,Sharon King,2019-01-11 19:34:50,4,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,2019-01-11 19:34:50,None
3,1.049701e+20,Veronica Pierce,2018-02-04 01:56:43,5,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,2018-02-04 01:56:43,None
4,1.105875e+20,Whitney Waldon Collier,2018-08-26 01:08:38,5,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,2018-08-26 01:08:38,None
...,...,...,...,...,...,...,...,...
1649995,1.155007e+20,Tabetha Hall,2020-06-27 19:23:32,5,We really had a great shopping experience! As ...,0x8889b57e39ea437b:0xe1fc31086a8e39a9,2020-06-27 19:23:32,None
1649996,1.070816e+20,Melissa Johnson,2021-02-12 23:38:19,5,Jessica was super helpful and kind. She led us...,0x8889b57e39ea437b:0xe1fc31086a8e39a9,2021-02-12 23:38:19,None
1649997,1.112208e+20,Blake Barnwell,2021-03-17 15:44:44,5,"Best place to find western boots, cowboy hats,...",0x8889b57e39ea437b:0xe1fc31086a8e39a9,2021-03-17 15:44:44,None
1649998,1.099382e+20,Danny Cassels,2021-03-23 06:45:24,4,Rediculas selection of boots .my favorite is t...,0x8889b57e39ea437b:0xe1fc31086a8e39a9,2021-03-23 06:45:24,None
